In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/SmallProject

/content/drive/MyDrive/SmallProject


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from data_util import *

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
                                   nn.MaxPool2d(kernel_size=2, stride=2),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64),
                                   nn.Dropout(p=0.05))
        
        self.conv2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                   nn.MaxPool2d(kernel_size=2, stride=2),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(128),
                                   nn.Dropout(p=0.05))
        
        self.pool1 = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
                                   nn.Flatten())
        
        self.fc1 = nn.Linear(128, 3)
        
        self.conv1.apply(init_weights)
        self.conv2.apply(init_weights)
        self.fc1.apply(init_weights)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.fc1(x)
        output = F.log_softmax(x, dim = 1)
        return output

In [5]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

In [10]:
def train(model, train_loader, epoch):
    optimizer = torch.optim.Adam(model.parameters())
    loss_func = torch.nn.CrossEntropyLoss()
    for epoch in range(epoch):
        model.train()
        print('epoch {}'.format(epoch + 1))
        train_loss = 0
        train_acc = 0
        for batch_idx, (images_data, target_labels) in enumerate(train_loader):
            images_data, target_labels = images_data.to(device), target_labels.to(device)
            optimizer.zero_grad()
            out = model.forward(images_data)
            target = torch.reshape(target_labels.argmax(dim=1, keepdim=True).long(),(-1,))
            loss = loss_func(out, target)
            train_loss += loss.item()
            pred = torch.reshape(out.argmax(dim=1,keepdim=True).long(),(-1,))
            train_correct = (pred == target).sum()
            train_acc += train_correct.item()
            loss.backward()
            optimizer.step()
        print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss / (len(ld_train)), train_acc / (len(ld_train))))

        model.eval()
        eval_loss = 0.
        eval_acc = 0.
        for batch_idx, (images_data, target_labels) in enumerate(val_loader):
            images_data, target_labels = images_data.to(device), target_labels.to(device)
            out = model.forward(images_data)
            target = ttorch.reshape(target_labels.argmax(dim=1, keepdim=True).long(),(-1,))
            eval_loss += loss_func(out, target).items()
            pred = torch.reshape(out.argmax(dim=1,keepdim=True).long(),(-1,))
            num_correct = (pred == target_labels).sum()
            eval_acc += num_correct.item()
        print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(ld_val)), eval_acc / (len(ld_val))))

In [11]:
def test(model, test_loader):
    model.eval()
    test_acc = 0.
    for batch_idx, (images_data, target_labels) in enumerate(test_loader):
        images_data, target_labels = images_data.to(device), target_labels.to(device)
        out = model.forward(images_data)
        target = target_labels.argmax(dim=1, keepdim=True).float()
        pred = torch.max(out, 1)[1]
        test_correct = (pred == target).sum()
        test_acc += train_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(test_loss / (len(ld_test)), test_acc / (len(ld_test))))

In [13]:
device = 'cuda'
ld_train = Lung_Train_Dataset()
ld_test = Lung_Test_Dataset()
ld_val = Lung_Val_Dataset()
bs_val = 64
train_loader = DataLoader(ld_train, batch_size = bs_val, shuffle = True)
test_loader = DataLoader(ld_test, batch_size = bs_val, shuffle = True)
val_loader = DataLoader(ld_val, batch_size = bs_val, shuffle = True)

model = Net().to(device)
train(model, train_loader, 10)
test(model, test_loader)

KeyboardInterrupt: ignored